In [53]:
import LowLevelFEM as FEM
using LowLevelFEM

gmsh.initialize()

In [54]:
gmsh.open("axi2D-2.geo")

Info    : Reading 'axi2D-2.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 60%] Meshing curve 5 (Line)
Info    : [ 80%] Meshing curve 6 (Line)
Info    : [ 90%] Meshing curve 7 (Line)
Info    : Done meshing 1D (Wall 0.000360992s, CPU 0.000347s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 50%] Meshing surface 2 (Transfinite)
Info    : Done meshing 2D (Wall 5.264e-05s, CPU 5e-05s)
Info    : 15 nodes 28 elements
Info    : Done reading 'axi2D-2.geo'


In [55]:
mat1 = FEM.material("body1", E=2.e5, ν=0.3)
mat2 = FEM.material("body2", E=2.e5, ν=0.3)
problem = FEM.Problem([mat1, mat2], type="AxiSymmetric", bandwidth="none")

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 5)


LowLevelFEM.Problem("axi2D-2", "AxiSymmetric", 2, [("body1", 200000.0, 0.3, 7.85e-9), ("body2", 200000.0, 0.3, 7.85e-9)], 1.0, 15)

In [56]:
#gmsh.fltk.run()

In [57]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [58]:
K = FEM.stiffnessMatrix(problem)

30×30 SparseArrays.SparseMatrixCSC{Float64, Int64} with 363 stored entries:
⎡⠛⣤⠀⠀⠀⠀⣿⣤⠀⠀⠀⠛⠀⣿⠀⎤
⎢⠀⠀⠛⣤⠀⠀⠀⠀⠛⣿⣤⠀⠀⠀⣿⎥
⎢⠀⠀⠀⠀⠛⣤⠀⣤⣤⠀⠛⠛⣿⣿⣿⎥
⎢⠛⣿⠀⠀⠀⣤⣿⣿⠀⠀⠀⠛⣤⣿⠀⎥
⎢⠀⠀⣿⣤⠀⠛⠀⠀⣿⣿⣤⠀⠛⠀⣿⎥
⎢⣤⠀⠀⠛⣿⠀⣤⠀⠀⠛⠓⣤⣿⣤⠛⎥
⎢⣤⣤⠀⠀⣿⣿⣤⣿⠛⠀⠛⣿⣿⣿⠛⎥
⎣⠀⠀⠛⠛⠛⠛⠀⠀⠛⠛⠛⠀⠛⠀⠛⎦

In [59]:
M = FEM.massMatrix(problem)

30×30 SparseArrays.SparseMatrixCSC{Float64, Int64} with 30 stored entries:
⎡⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⎦

In [60]:
supp = FEM.displacementConstraint("supp", uy=0)
supp2 = FEM.displacementConstraint("load", uy=0.0001)
load = FEM.load("load", fx=0.001, fy=1)

("load", 0.001, 1, 0)

In [61]:
f = FEM.loadVector(problem, [load])

30-element Vector{Float64}:
   0.0
   0.0
   0.0
   0.0
   0.2879793265790645
 287.9793265790645
   0.1832595714594045
 183.25957145940447
   0.0
   0.0
   ⋮
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0

In [62]:
FEM.applyBoundaryConditions!(problem, K, f, [supp])

In [63]:
q = FEM.solveDisplacement(K, f)

30-element Vector{Float64}:
 -1.5030922415197964e-5
  0.0
 -3.002550724844994e-5
  0.0
 -2.978597623883016e-5
  9.98873058636508e-5
 -1.4753679882901774e-5
  0.00010008841052820427
 -1.4978169334560818e-5
  5.00701699199387e-5
  ⋮
  7.509798430097896e-5
 -1.5019232232159304e-5
  2.5034385772055184e-5
 -2.2486569287257476e-5
  5.0007176650063396e-5
 -2.2522883208703935e-5
  2.500467877836452e-5
 -2.2408238932206295e-5
  7.500392469137584e-5

In [64]:
S = FEM.solveStress(problem, q)

LowLevelFEM.StressField([[0.00032219739849290163; 0.00017984896982550848; … ; 0.0; 0.00019147364595539784;;], [0.000407286612302854; 0.0001747062142590867; … ; 0.0; 0.001219691650751309;;], [0.00025985582385595896; 0.0001564010484018923; … ; 0.0; -0.0001303804410069942;;], [0.000283819864447421; 0.00014622369230693044; … ; 0.0; 0.00044494756753321596;;], [0.0004414574800232496; 0.0003227490874791514; … ; 0.0; 0.003166366611555671;;], [0.0002437824094488339; 0.0007147366383763147; … ; 0.0; 0.005898702898659847;;], [0.0003694953412559654; 0.0003190110333445031; … ; 0.0; 0.0015567927901792289;;], [0.0002988470420785261; 0.000671515322579748; … ; 0.0; 0.003417976337485973;;]], [5, 6, 7, 8, 9, 10, 11, 12], 1)

In [65]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)

In [66]:
u = FEM.showDoFResults(problem, q, "uvec", name="uvec", visible=false)
ux = FEM.showDoFResults(problem, q, "ux", name="ux", visible=false)
uy = FEM.showDoFResults(problem, q, "uy", name="uy", visible=false)
uz = FEM.showDoFResults(problem, q, "uz", name="uz", visible=false)
s = FEM.showStressResults(problem, S, "s", name="σ smooth", visible=true, smooth=true)
ss = FEM.showStressResults(problem, S, "s", name="σ", visible=false, smooth=false)
sx = FEM.showStressResults(problem, S, "sx", name="σx", visible=false, smooth=true)
sy = FEM.showStressResults(problem, S, "sy", name="σy", visible=false, smooth=true)
sz = FEM.showStressResults(problem, S, "sz", name="σz", visible=false, smooth=true)
sxy = FEM.showStressResults(problem, S, "sxy", name="τxy", visible=false, smooth=true)
syz = FEM.showStressResults(problem, S, "syz", name="τyz", visible=false, smooth=true)
szx = FEM.showStressResults(problem, S, "szx", name="τzx", visible=false, smooth=true)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


11

In [67]:
FEM.plotOnPath(problem, "path", sx, 100, name="σx", visible=false);
FEM.plotOnPath(problem, "path", sxy, 100, name="τxy", visible=false);
FEM.plotOnPath(problem, "path", ux, 100, name="ux", visible=false);

Error   : Physical name 'path' does not exist


ErrorException: Physical name 'path' does not exist

In [68]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [69]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.2
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.6.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [70]:
gmsh.finalize()